<a href="https://colab.research.google.com/github/arthurvaz05/Python.Script/blob/master/Golden_Mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install rpy2

In [0]:
%load_ext rpy2.ipython

In [1]:
import pandas as pd
import numpy as np
import rpy2
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 5.1MB/s 


In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#verificar os arquivos disponiveis na pasta
c = !ls "/content/drive/My Drive/CCARQSOLAR/Volume"

In [0]:
li = []
for num in list(range(0,len(c),1)):
  li.append(c[num].split('  '))
li = sum(li,[])

In [0]:
x = pd.Series(li).str.contains('.csv').tolist()
x = [i for i, x in enumerate(x) if x]
t = [li[i] for i in x]
k = [x.replace(' ', '') for x in t]

In [0]:
def f(x):
  x.dropna(inplace=True)
  x.drop_duplicates(inplace=True)
  return x

In [0]:
def f2(x):
  for num in list(range(0,len(x),1)):
    if num ==0:
      mes_final = mes[num]
    else:
      mes_final = pd.concat([mes_final, mes[num]])
  return mes_final   

In [23]:
list_mes = pd.Series(k)
mes = list_mes.map(lambda x: pd.read_csv('/content/drive/My Drive/CCARQSOLAR/Volume/'+x, delimiter=";",encoding='latin-1'))
mes = mes.apply(f)
mes = f2(mes)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  


In [0]:
import datetime as dt
mes['Data'] = pd.to_datetime(mes['Data'])

base_2018 = mes[mes['Data'].dt.year == 2018]
base_julho = mes[mes['Data'].dt.year != 2018]

In [0]:
clientes = pd.read_csv('/content/drive/My Drive/CCARQSOLAR/Volume/Meta Base/Diimensao Cliente v3.csv', delimiter=";",encoding='latin-1')

In [0]:
#SEPARAR AS CIDADES DOS ESTADOS EM DUAS COLUNAS
#teste = clientes['CIDADE'].map(lambda x: x.split('/'))
#clientes['ESTADO'] = teste.map(lambda x: x[1])
#clientes['CIDADE'] = teste.map(lambda x: x[0])

In [0]:
#CARREGAR AS BASES DE DADOS DEMOGRAFICOS E SOCIAIS DAS REGIOES
ipc_maps = pd.read_csv('/content/drive/My Drive/CCARQSOLAR/IPC Maps/IPCMaps_2018_Bairro_ADJ.csv', delimiter=";",encoding='latin-1')
ipc_maps_2 = pd.read_csv('/content/drive/My Drive/CCARQSOLAR/IPC Maps/ipc2018_nao_alcoolicas_bairros_ADJ.csv', delimiter=";",encoding='latin-1')

In [0]:
df = pd.DataFrame( {'name':pd.Series(clientes['CIDADE']).unique() } )

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['name'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

def separar_lat(x):
  lat,long,x = x
  return lat

def separar_long(x):
  lat,long,x = x
  return long

clientes['lat'] = df['point'].apply(separar_lat)
clientes['long'] = df['point'].apply(separar_long)

In [0]:
clientes['CIDADE'] = clientes['CIDADE'].map(lambda x: x.upper())
clientes['BAIRRO'] = clientes['BAIRRO'].map(lambda x: x.replace(" ",""))
clientes['CIDADE'] = clientes['CIDADE'].map(lambda x: x.replace(" ",""))
ipc_maps['MUNICIPIO'] = ipc_maps['MUNICIPIO'].map(lambda x: x.replace(" ",""))
ipc_maps['BAIRRO'] = ipc_maps['BAIRRO'].map(lambda x: x.replace(" ",""))
#CRIAR A CHAVE PARA JOIN DAS BASES DE CLIENTES E INFO DAS REGIOES
ipc_maps['KEY'] = ipc_maps['BAIRRO'] +'_'+ ipc_maps['MUNICIPIO'] + '_' + ipc_maps['UF']
clientes['KEY'] = clientes['BAIRRO'] + '_' + clientes['CIDADE'] + '_' + clientes['ESTADO']
cluster = pd.merge(clientes,ipc_maps,on ='KEY')

In [0]:
#TRANSFORMAR OS NUMEROS DE CHECKOUT EM DUMMIES
dummy = pd.get_dummies(cluster['SUBCANAL'])
cluster = pd.concat([cluster, dummy], axis=1)

In [0]:
def replace_comma(x):
    try: 
        return x.replace(',', '.')
    except AttributeError:
        return x

In [0]:
cluster.fillna(0, inplace = True)
var = cluster.loc[:,['LITORAL','INTERIOR','CAPITAL',
       'Sexo_Masculina', 'Sexo_Feminina', 'Masc_0.a.17', 'Masc_18.a.24',
       'Masc_25.a.29', 'Masc_30.a.39', 'Masc_40.e.mais', 'Fem_0.a.17',
       'Fem_18.a.24', 'Fem_25.a.29', 'Fem_30.a.39', 'Fem_40.e.mais',
       'Domicilios_Urbanos', 'Domicilios_Rurais',
       'CLASSE_A', 'CLASSE_B', 'CLASSE_C', 'CLASSE_D', 'Consumo_Urbano',
       'Consumo_Rural', 'IPC', 'Densidade.Demográfica', 'ALIMENTACAO',
       'COMERCIO.ATACADISTA', 'COMERCIO.VAREJISTA',
       'Alimentacao_Domicilio_CLASSE_A', 'Alimentacao_Domicilio_CLASSE_B',
       'Alimentacao_Domicilio_CLASSE_C', 'Alimentacao_Domicilio_CLASSE_D',
       'Alimentacao_Domicilio_CLASSE_TOTAL',
       'Alimentacao_FORA_Domicilio_CLASSE_A',
       'Alimentacao_FORA_Domicilio_CLASSE_B',
       'Alimentacao_FORA_Domicilio_CLASSE_C',
       'Alimentacao_FORA_Domicilio_CLASSE_D',
       'Alimentacao_FORA_Domicilio_CLASSE_TOTAL', 'BEBIDAS_CLASSE_A',
       'BEBIDAS_CLASSE_B', 'BEBIDAS_CLASSE_C', 'BEBIDAS_CLASSE_D',
       'BEBIDAS_CLASSE_TOTAL', 'DESPESA_TOTAL_CLASSE_A',
       'DESPESA_TOTAL_CLASSE_B', 'DESPESA_TOTAL_CLASSE_C',
       'DESPESA_TOTAL_CLASSE_D', 'DESPESA_TOTAL_CLASSE_TOTAL', 'CONSUMO_RURAL',
       'CONSUMO_TOTAL', 'COMPRA CENTRALIZADA', 'HIPERMERCADO PARA ASSOCIADOS',
       'HIPERMERCADOS', 'SUPERM ASSO 10 A 19 CHECK-OUTS',
       'SUPERM ASSO 20 A 29 CHECK-OUTS', 'SUPERM ASSO 5 A 9 CHECK-OUTS',
       'SUPERM DE 10 A 19 CHECK-OUTS', 'SUPERM DE 20 A 29 CHECK-OUTS',
       'SUPERM DE 30 A 50 CHECK-OUTS', 'SUPERM DE 5 A 9 CHECK-OUTS']]
var = var.applymap(lambda x: replace_comma(x))

In [0]:
x = var.values
y = cluster.loc[:,['RAZAO_SOCIAL']].values

In [0]:
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(x)
from sklearn.decomposition import PCA
pca = PCA()

In [0]:
principalComponents = pca.fit_transform(x)

In [0]:
principalComponents_2d = pd.DataFrame(principalComponents)
principalComponents_2d.index = cluster.index
#principalComponents_2d.columns = []
principalComponents_2d.head()

In [0]:
print(pca.explained_variance_ratio_) 

In [0]:
sum(pca.explained_variance_ratio_[0:23])

In [0]:
pca = PCA(n_components=23)
principalComponents = pca.fit_transform(x)
principalComponents_2d = pd.DataFrame(principalComponents)
principalComponents_2d.index = cluster.index
principalComponents_2d.columns = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12','PC13','PC14','PC15','PC16','PC17','PC18','PC19','PC20','PC21','PC22','PC23']
principalComponents_2d.head()

In [0]:
sum(pca.explained_variance_ratio_)

0.8012513423717325

In [0]:
principalDf = pd.DataFrame(principalComponents)

In [0]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6)

clusters = kmeans.fit(principalDf)

In [0]:
resultado = pd.concat([cluster.loc[:,['RAZAO_SOCIAL']], var], axis=1)
resultado['Cluster'] = clusters.labels_

In [0]:
resultado.head()

,RAZAO_SOCIAL,LITORAL,INTERIOR,CAPITAL,Sexo_Masculina,Sexo_Feminina,Masc_0.a.17,Masc_18.a.24,Masc_25.a.29,Masc_30.a.39,Masc_40.e.mais,Fem_0.a.17,Fem_18.a.24,Fem_25.a.29,Fem_30.a.39,Fem_40.e.mais,Domicilios_Urbanos,Domicilios_Rurais,CLASSE_A,CLASSE_B,CLASSE_C,CLASSE_D,Consumo_Urbano,Consumo_Rural,IPC,Densidade.Demográfica,ALIMENTACAO,COMERCIO.ATACADISTA,COMERCIO.VAREJISTA,Alimentacao_Domicilio_CLASSE_A,Alimentacao_Domicilio_CLASSE_B,Alimentacao_Domicilio_CLASSE_C,Alimentacao_Domicilio_CLASSE_D,Alimentacao_Domicilio_CLASSE_TOTAL,Alimentacao_FORA_Domicilio_CLASSE_A,Alimentacao_FORA_Domicilio_CLASSE_B,Alimentacao_FORA_Domicilio_CLASSE_C,Alimentacao_FORA_Domicilio_CLASSE_D,Alimentacao_FORA_Domicilio_CLASSE_TOTAL,BEBIDAS_CLASSE_A,BEBIDAS_CLASSE_B,BEBIDAS_CLASSE_C,BEBIDAS_CLASSE_D,BEBIDAS_CLASSE_TOTAL,DESPESA_TOTAL_CLASSE_A,DESPESA_TOTAL_CLASSE_B,DESPESA_TOTAL_CLASSE_C,DESPESA_TOTAL_CLASSE_D,DESPESA_TOTAL_CLASSE_TOTAL,CONSUMO_RURAL,CONSUMO_TOTAL,COMPRA CENTRALIZADA,HIPERMERCADO PARA ASSOCIADOS,HIPERMERCADOS,SUPERM ASSO 10 A 19 CHECK-OUTS,SUPERM ASSO 20 A 29 CHECK-OUTS,SUPERM ASSO 5 A 9 CHECK-OUTS,SUPERM DE 10 A 19 CHECK-OUTS,SUPERM DE 20 A 29 CHECK-OUTS,SUPERM DE 30 A 50 CHECK-OUTS,SUPERM DE 5 A 9 CHECK-OUTS,Cluster
0,MR SUPERMERCADOS LTDA,0.0,1.0,0.0,4644,5216,927,1800,1200,655,2077,909,1940,1393,676,2665,3839,1,301,701,1631,1124,5663,1,0.00106,3892,177,1,1028,4036,1054,2761,498,6226,3966,7421,6918,3564,2879,3359,2391,2319,7176,5800,4019,2333,996,5214,5713,1,5747,0,0,0,0,0,0,1,0,0,0,5
1,ATACADAO BOM E BARATO SUPERMER,0.0,1.0,0.0,434,637,1836,640,436,1123,2849,1803,884,428,1195,3539,272,1,505,1150,2563,1611,4951,1,0.00194,4642,154,510,52,5265,4115,5183,2703,834,14,687,620,5729,5249,4669,4426,4591,1793,8033,977,4310,3275,630,7838,1,7896,0,0,0,0,0,0,0,0,0,1,2
2,BOMPRECO SUPERMERCADOS DO NORD,0.0,1.0,0.0,434,637,1836,640,436,1123,2849,1803,884,428,1195,3539,272,1,505,1150,2563,1611,4951,1,0.00194,4642,154,510,52,5265,4115,5183,2703,834,14,687,620,5729,5249,4669,4426,4591,1793,8033,977,4310,3275,630,7838,1,7896,0,0,0,0,0,1,0,0,0,0,2
3,ATACADAO BOM E BARATO SUPERMER,0.0,1.0,0.0,434,637,1836,640,436,1123,2849,1803,884,428,1195,3539,272,1,505,1150,2563,1611,4951,1,0.00194,4642,154,510,52,5265,4115,5183,2703,834,14,687,620,5729,5249,4669,4426,4591,1793,8033,977,4310,3275,630,7838,1,7896,0,0,0,0,0,0,0,0,0,1,2
4,BOMPRECO SUPERMERCADOS DO NORD,0.0,1.0,0.0,434,637,1836,640,436,1123,2849,1803,884,428,1195,3539,272,1,505,1150,2563,1611,4951,1,0.00194,4642,154,510,52,5265,4115,5183,2703,834,14,687,620,5729,5249,4669,4426,4591,1793,8033,977,4310,3275,630,7838,1,7896,0,0,0,0,0,0,1,0,0,0,2


In [0]:
from google.colab import files
resultado.to_csv('filename.csv') 
files.download('filename.csv')